<a href="https://colab.research.google.com/github/ayz1070/CapstoneStockModelingProject/blob/main/LSTM_%EB%AA%A8%EB%8D%B8_%EC%A0%81%EC%9A%A9(%EC%88%98%EC%A0%95v6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

In [4]:
# Colab => 데이터 CSV 파일 가져오기
from google.colab import files
uploaded = files.upload()

Saving kospi100.xlsx to kospi100.xlsx


In [5]:
kospi = pd.read_excel("kospi100.xlsx", engine='openpyxl')

companies_list = []
companies = kospi['종목명'][:2].tolist()
companies

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['삼성전자', 'LG에너지솔루션']

In [6]:
# sequence_dataset 만들기
def make_sequence_dataset(feature, label, window_size):

  feature_list = []   # 생성될 feature list
  label_list = []     # 생성될 label list

  for i in range(len(feature)-window_size): # range는 전체값에서 window_size를 뺀 값

    feature_list.append(feature[i:i+window_size]) # feature list 에 i번째서 부터 window size 만큼의 입력데이터를 추가
    label_list.append(label[i+window_size]) # label list 에 그 다음 번째('window_size + 1' 번째)의 정답데이터를 추가

  return np.array(feature_list), np.array(label_list)

In [7]:
#예측값 저장 dict 만들기

day_five = {}
day_ten = {}

In [16]:
for company in companies:
 print(company, '주가 분석중...')
 csv_df = pd.read_csv(f'{company}.csv')  # CSV 파일명 입력
 csv_df = csv_df.loc[:, ['Date', 'Close', 'BPS', 'EPS', '금리', '환율']]
 csv_df = csv_df.set_index('Date') # 날짜를 index로 바꿈
 csv_df.dropna()

 # 날짜 빼고 정규화 작업 진행 -> MinMaxScaler 사용
 scaler = MinMaxScaler()

 # 정규화 수행 -> 정규화된 데이터프레임은 scaled_df
 scale_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
 scaled_df = scaler.fit_transform(csv_df[scale_cols]) # 정규화된 데이터는 넘파이 형태
 scaled_df = pd.DataFrame(scaled_df, columns = scale_cols) # Pandas DataFrame 형태로 변경
 scaled_csv_df = scaled_df.set_index(csv_df.index)

 # 입력데이터 -> 이전날까지의 종가와 변수들
 # 정답데이터 -> 다음날의 종가
 feature_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
 label_cols = ['Close']

 # 입력데이터, 정답데이터 프레임 -> feature_df, label_df
 feature_df = scaled_csv_df[feature_cols]
 label_df = scaled_csv_df[label_cols]

 # DataFrame을 Numpy 형태로 저장
 feature_np = feature_df.to_numpy()
 label_np = label_df.to_numpy()

 window_size = 80  # window_size 만큼의 입력데이터를 이용해 바로 다음 값에 오는 Close 값을 예측
 X, y = make_sequence_dataset(feature_np, label_np, window_size) # X에는 np.array(feature_list), y에는 np.array(label_list) 가 대입됨

 # 모델 훈련을 위한 준비 -> 훈련을 하고, 제대로 훈련이 됐는지 테스트
 split = int(len(X)*0.7) # 테스트 데이터로 분리 -> train:test = 7:3

 # 훈련data 는 전체 데이터의 70퍼센트
 X_train = X[0:split]
 y_train = y[0:split]

 # 테스트data 는 전체 데이터의 30퍼센트
 X_test = X[split:]
 y_test = y[split:]

 ## 5일 예측
 #LSTM 모델 구축
 model = Sequential()

 # 1차원 feature map 생성
 model.add(Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

 # LSTM layer
 model.add(LSTM(units = 16, activation = 'tanh'))
 model.add(Dense(units = 16, activation = 'sigmoid'))
 model.add(Dense(units = 5)) # 출력층

 # 모델 컴파일
 loss = Huber()
 optimizer = Adam(0.0005)
 model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

 # 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
 earlystopping = EarlyStopping(monitor='loss', patience=10)


 # 모델 학습 -> epoch은 100번 진행
 model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

 # 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
 predictions = model.predict(X_test)

 # 실제값 변환
 # 실제값으로 변화시키기 위해서 기존 데이터에서 종가의 최댓값과 최솟값을 가져옵니다
 close_min = csv_df['Close'].min()
 close_max = csv_df['Close'].max()

 # MinMaxScaler이용해서 실제값으로 역변환
 scaler2 = MinMaxScaler()
 scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

 # MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
 scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
 scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

 # 예측한 출력값을 실제값으로 역변환
 original_pred_values = scaler2.inverse_transform(predictions)

 # 실제로 에측하기
 # feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
 pred_feature = feature_df.tail(window_size)

 pred_feature_list = []
 pred_feature_list.append(pred_feature)
 pred_feature = np.array(pred_feature_list)

 # 5일 예측 -> 5일까지의 예측값 5개 출력
 predictions_5d = model.predict(pred_feature)
 pred_values_5d = scaler2.inverse_transform(predictions_5d)


 ##10일 예측
 #LSTM 모델 구축
 model = Sequential()

 # 1차원 feature map 생성
 model.add(Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

 # LSTM layer
 model.add(LSTM(units = 16, activation = 'tanh'))
 model.add(Dense(units = 16, activation = 'sigmoid'))
 model.add(Dense(units = 10)) # 출력층

  # 모델 컴파일
 loss = Huber()
 optimizer = Adam(0.0005)
 model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

 # 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
 earlystopping = EarlyStopping(monitor='loss', patience=10)


 # 모델 학습 -> epoch은 100번 진행
 model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

 # 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
 predictions = model.predict(X_test)

 # 실제값 변환
 # 실제값으로 변화시키기 위해서 기존 데이터에서 종가의 최댓값과 최솟값을 가져옵니다
 close_min = csv_df['Close'].min()
 close_max = csv_df['Close'].max()

 # MinMaxScaler이용해서 실제값으로 역변환
 scaler2 = MinMaxScaler()
 scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

 # MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
 scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
 scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

 # 예측한 출력값을 실제값으로 역변환
 original_pred_values = scaler2.inverse_transform(predictions)

 # 실제로 에측하기
 # feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
 pred_feature = feature_df.tail(window_size)

 pred_feature_list = []
 pred_feature_list.append(pred_feature)
 pred_feature = np.array(pred_feature_list)

 # 10일 예측 -> 10일까지의 예측값 10개 출력
 predictions_10d = model.predict(pred_feature)
 pred_values_10d = scaler2.inverse_transform(predictions_10d)

 #pred_values_5d = pred_values_5d.flatten().tolist()
 #pred_values_10d = pred_values_10d.flatten().tolist()

 close_5df = csv_df[['Close']] # 종가데이터
 close_5df.reset_index(inplace=True)
 close_5df.columns = ['Date', 'Close']  # 열 이름 변경
 close_5df['Date'] = pd.to_datetime(close_5df['Date'])  # Date 열을 날짜로 변환

 close_10df = csv_df[['Close']] # 종가데이터
 close_10df.reset_index(inplace=True)
 close_10df.columns = ['Date', 'Close']  # 열 이름 변경
 close_10df['Date'] = pd.to_datetime(close_10df['Date'])  # Date 열을 날짜로 변환

 pred5dCsvDf = pd.DataFrame(columns = ['Date', 'Close'])
 pred10dCsvDf = pd.DataFrame(columns = ['Date', 'Close'])

# 5일 예측 날짜와 종가
 for i in range (len(pred_values_5d[0])):
  # 마지막 날짜를 찾고 그 다음 날짜 계산
  last_date = close_5df['Date'].max()
  next_date = last_date + pd.DateOffset(days=1)  # 하루를 더해 다음 날짜 계산
  # 새로운 데이터 생성 및 추가
  new_data = {'Date': [next_date], 'Close': [pred_values_5d[0,i]]}
  new_df = pd.DataFrame(new_data)
  pred5dCsvDf = pd.concat([pred5dCsvDf, new_df], ignore_index=True)
  # 그 다음 날짜 계산을 위한 마지막 날짜 변경
  close_5df = pd.concat([close_5df, new_df], ignore_index=True)

# 10일 예측 날짜와 종가
 for i in range (len(pred_values_10d[0])):
  # 마지막 날짜를 찾고 그 다음 날짜 계산
  last_date = close_10df['Date'].max()
  next_date = last_date + pd.DateOffset(days=1)  # 하루를 더해 다음 날짜 계산
  # 새로운 데이터 생성 및 추가
  new_data = {'Date': [next_date], 'Close': [pred_values_10d[0,i]]}
  new_df = pd.DataFrame(new_data)
  pred10dCsvDf = pd.concat([pred10dCsvDf, new_df], ignore_index=True)
  # 그 다음 날짜 계산을 위한 마지막 날짜 변경
  close_10df = pd.concat([close_10df, new_df], ignore_index=True)

# Date 열을 문자열로 다시 변환 (예: 'YYYY-MM-DD' 형식)
 pred5dCsvDf['Date'] = pred5dCsvDf['Date'].dt.strftime('%Y-%m-%d')
 pred10dCsvDf['Date'] = pred10dCsvDf['Date'].dt.strftime('%Y-%m-%d')

 day_five[company] = pred5dCsvDf.values.tolist()
 day_ten[company] = pred10dCsvDf.values.tolist()

 print('5일 예측값', pred_values_5d)
 print('10일 예측값', pred_values_10d)
 print('/n')

삼성전자 주가 분석중...
Epoch 1/100
45/45 [==============================] - 3s 20ms/step - loss: 0.0743 - mse: 0.1488
Epoch 2/100
45/45 [==============================] - 1s 20ms/step - loss: 0.0189 - mse: 0.0378
Epoch 3/100
45/45 [==============================] - 1s 20ms/step - loss: 0.0082 - mse: 0.0165
Epoch 4/100
45/45 [==============================] - 1s 30ms/step - loss: 0.0061 - mse: 0.0121
Epoch 5/100
45/45 [==============================] - 1s 24ms/step - loss: 0.0042 - mse: 0.0084
Epoch 6/100
45/45 [==============================] - 1s 20ms/step - loss: 0.0030 - mse: 0.0061
Epoch 7/100
45/45 [==============================] - 1s 20ms/step - loss: 0.0024 - mse: 0.0048
Epoch 8/100
45/45 [==============================] - 1s 20ms/step - loss: 0.0020 - mse: 0.0040
Epoch 9/100
45/45 [==============================] - 1s 20ms/step - loss: 0.0016 - mse: 0.0032
Epoch 10/100
45/45 [==============================] - 1s 19ms/step - loss: 0.0013 - mse: 0.0026
Epoch 11/100
45/45 [==============

<ipython-input-16-1dda82ab26e1>:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_5df['Date'] = pd.to_datetime(close_5df['Date'])  # Date 열을 날짜로 변환
<ipython-input-16-1dda82ab26e1>:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_10df['Date'] = pd.to_datetime(close_10df['Date'])  # Date 열을 날짜로 변환


Epoch 1/100
7/7 [==============================] - 2s 21ms/step - loss: 0.1123 - mse: 0.2246
Epoch 2/100
7/7 [==============================] - 0s 21ms/step - loss: 0.0942 - mse: 0.1884
Epoch 3/100
7/7 [==============================] - 0s 20ms/step - loss: 0.0783 - mse: 0.1565
Epoch 4/100
7/7 [==============================] - 0s 21ms/step - loss: 0.0635 - mse: 0.1271
Epoch 5/100
7/7 [==============================] - 0s 20ms/step - loss: 0.0513 - mse: 0.1026
Epoch 6/100
7/7 [==============================] - 0s 21ms/step - loss: 0.0424 - mse: 0.0848
Epoch 7/100
7/7 [==============================] - 0s 21ms/step - loss: 0.0360 - mse: 0.0719
Epoch 8/100
7/7 [==============================] - 0s 21ms/step - loss: 0.0313 - mse: 0.0626
Epoch 9/100
7/7 [==============================] - 0s 22ms/step - loss: 0.0284 - mse: 0.0568
Epoch 10/100
7/7 [==============================] - 0s 24ms/step - loss: 0.0260 - mse: 0.0521
Epoch 11/100
7/7 [==============================] - 0s 21ms/step - lo

<ipython-input-16-1dda82ab26e1>:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_5df['Date'] = pd.to_datetime(close_5df['Date'])  # Date 열을 날짜로 변환
<ipython-input-16-1dda82ab26e1>:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_10df['Date'] = pd.to_datetime(close_10df['Date'])  # Date 열을 날짜로 변환




---



In [17]:
day_five

{'삼성전자': [['2023-08-09', 63790.03515625],
  ['2023-08-10', 66688.203125],
  ['2023-08-11', 66924.765625],
  ['2023-08-12', 66629.515625],
  ['2023-08-13', 66841.5390625]],
 'LG에너지솔루션': [['2023-08-09', 517797.46875],
  ['2023-08-10', 544980.5625],
  ['2023-08-11', 536391.5625],
  ['2023-08-12', 519790.625],
  ['2023-08-13', 528428.6875]]}

In [18]:
day_ten

{'삼성전자': [['2023-08-09', 59673.3046875],
  ['2023-08-10', 57523.9609375],
  ['2023-08-11', 54206.4140625],
  ['2023-08-12', 55382.55859375],
  ['2023-08-13', 58611.7421875],
  ['2023-08-14', 54988.73828125],
  ['2023-08-15', 57231.546875],
  ['2023-08-16', 61228.921875],
  ['2023-08-17', 59582.48046875],
  ['2023-08-18', 56510.80078125]],
 'LG에너지솔루션': [['2023-08-09', 544863.5],
  ['2023-08-10', 523724.65625],
  ['2023-08-11', 544823.5],
  ['2023-08-12', 535101.125],
  ['2023-08-13', 505000.6875],
  ['2023-08-14', 540643.125],
  ['2023-08-15', 542305.1875],
  ['2023-08-16', 526651.875],
  ['2023-08-17', 523122.46875],
  ['2023-08-18', 537933.75]]}

In [ ]:
import requests

data = {
    'day_five': day_five
}

server_url = 'https://eb86-39-118-146-59.ngrok-free.app/day_five'

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

성공


In [ ]:
import requests

data = {
    'day_ten': day_ten
}

server_url = 'https://eb86-39-118-146-59.ngrok-free.app/day_ten'

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

성공
